In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from flask import Flask, render_template, request
from PIL import Image


In [2]:
dataset_path = 'dataset_mouth'
saved_model_path = 'model_mouth.h5'


In [3]:
image_size = (64, 64)
batch_size = 32
epochs = 10


In [4]:
# Create the mouth detection model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [5]:
def preprocess_dataset(dataset_path):
    # Preprocess the dataset by resizing images and creating label directories
    open_dir = os.path.join(dataset_path, 'class0')
    closed_dir = os.path.join(dataset_path, 'class1')

    if not os.path.exists(open_dir):
        os.makedirs(open_dir)
    if not os.path.exists(closed_dir):
        os.makedirs(closed_dir)

    for filename in os.listdir(dataset_path):
        if filename.endswith('.jpg'):
            if 'class0' in filename:
                image_dir = open_dir
            else:
                image_dir = closed_dir

            img = cv2.imread(os.path.join(dataset_path, filename))
            img = cv2.resize(img, image_size)
            cv2.imwrite(os.path.join(image_dir, filename), img)

preprocess_dataset(dataset_path)

In [6]:
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)




Found 4096 images belonging to 2 classes.
Found 1023 images belonging to 2 classes.


In [7]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[ModelCheckpoint(saved_model_path, save_best_only=True)]
)

Epoch 1/10
128/128 [==============================] - 19s 133ms/step - loss: 0.2649 - accuracy: 0.8877 - val_loss: 0.3384 - val_accuracy: 0.8427
Epoch 2/10
128/128 [==============================] - 12s 95ms/step - loss: 0.1437 - accuracy: 0.9487 - val_loss: 0.2230 - val_accuracy: 0.9022
Epoch 3/10
128/128 [==============================] - 12s 90ms/step - loss: 0.1121 - accuracy: 0.9622 - val_loss: 0.1875 - val_accuracy: 0.9284
Epoch 4/10
128/128 [==============================] - 12s 90ms/step - loss: 0.0830 - accuracy: 0.9751 - val_loss: 0.1698 - val_accuracy: 0.9345
Epoch 5/10
128/128 [==============================] - 11s 87ms/step - loss: 0.0711 - accuracy: 0.9771 - val_loss: 0.1930 - val_accuracy: 0.9284
Epoch 6/10
128/128 [==============================] - 11s 87ms/step - loss: 0.0647 - accuracy: 0.9797 - val_loss: 0.2480 - val_accuracy: 0.9103
Epoch 7/10
128/128 [==============================] - 11s 88ms/step - loss: 0.0470 - accuracy: 0.9841 - val_loss: 0.2775 - val_accuracy

In [9]:
model.evaluate(validation_generator)

32/32 [==============================] - 2s 75ms/step - loss: 0.1958 - accuracy: 0.9482


[0.1957891285419464, 0.9481915831565857]

In [10]:
model_json = model.to_json()
with open("model_mouth.json","w") as json_file:
    json_file.write(model_json)